# Wordle Solver Algorithm

Wordle is a popular word game hosted by the New York Times,
[https://www.nytimes.com/games/wordle/index.html](https://www.nytimes.com/games/wordle/index.html)

<img src="https://static01.nyt.com/images/2022/01/31/crosswords/wordle-art/wordle-art-mediumSquareAt3X.jpg" alt="Drawing" style="width: 400px;"/>

The object of the game is to guess the day's 5-Letter word within 6 tries.  After each try the game provides feedback about how close your guess was to the target word.  Green letters are correct guesses, Yellow letters signify that the letter appears in the word but not in that location, and Gray letters are letters which are not in the word at all.  Using this feedback you can make better and better guesses until you correctly guess the target word.  In the hardmode variant of the game once a letter is correctly guessed (either yellow or green) you must use those letters in all subsequent guesses.  For example suppose that the word that we are trying to guess is HOUSE and for our first guess we guess SAUCE.

In [150]:
%%HTML
<style>
td , th {font-size: 50px}
</style>

<style>
td , th {font-size: 50px}
</style>
<table>
<thead>
<tr>
<th> H </th>
<th> O </th>
<th> U </th>
<th> S </th>
<th> E </th>
</tr>
</thead>
<tbody>
<tr>
<td><p style="background-color:yellow">S</p></td>
<td><p style="background-color:gray">A</p></td>
<td><p style="background-color:green">U</p></td>
<td><p style="background-color:gray">C</p></td>
<td><p style="background-color:green">E</p></td>
</tr>
</tbody>
</table>

Our guess of SAUCE incorrectly guessed A and C, correctly guessed the letters and locations for U and E, and correctly guessed the letter but incorrectly guessed the location for the letter S.  In hard mode all future guesses must contain the correctly guessed letters of S, U, and E and the U and E must remain in those locations.  For example MOUSE would be a permitted next guess but ELUDE would not.  The goal of the game is to continue guessing in such a manner until either the word is correctly guessed or the player has exhausted all 6 guesses.  Continuing our game might looks like:

<table>
<thead>
<tr>
<th> H </th>
<th> O </th>
<th> U </th>
<th> S </th>
<th> E </th>
</tr>
</thead>
<tbody>
<tr>
<td><p style="background-color:yellow">S</p></td>
<td><p style="background-color:gray">A</p></td>
<td><p style="background-color:green">U</p></td>
<td><p style="background-color:gray">C</p></td>
<td><p style="background-color:green">E</p></td>
</tr>
<tr>
<td><p style="background-color:gray">M</p></td>
<td><p style="background-color:green">O</p></td>
<td><p style="background-color:green">U</p></td>
<td><p style="background-color:green">S</p></td>
<td><p style="background-color:green">E</p></td>
</tr>
<tr>
<td><p style="background-color:green">H</p></td>
<td><p style="background-color:green">O</p></td>
<td><p style="background-color:green">U</p></td>
<td><p style="background-color:green">S</p></td>
<td><p style="background-color:green">E</p></td>
</tr>
</tbody>
</table>

Here we correctly guessed the solution in 3 guesses!  An interesting question then is: Can we create an algorithm to solve any wordle puzzle in as few guesses as possible?

# Wordle Solver Version 1
The first thing that we need to do is import a dictionary of 5-letter words, we will start with a common dictionary of words from a text file for convinience with the knowledge that it contains proper nouns, as well as may contain other entries which wordle will not accept as a viable word guess:

In [162]:
with open('WordleDict.txt', 'r') as file:
    word_list=[word for line in file for word in line.split()]

import string
alphabet_string = string.ascii_uppercase
alphabet_list = list(alphabet_string)

#Collect all 5 letter non-proper noun words from the dictionary
fiveletterwords = [x for x in word_list if len(x)==5 and x[0] not in alphabet_list]

Now that we have a list of five letter words we can consider a solving strategy.  One initial observation is that many of the words in the list will share letters with one another, a word like CHASE is much more likely to give us useful solving information than a word like WHIZZ as letters like C,H, and S are much more common than W or Z.  In order to take this letter frequency information into account we need to split the words up into their component characters and count how many times that each character appears in the list.

In [163]:
#collect lists of each letter in each position
l_1 = [x[0] for x in fiveletterwords]
l_2 = [x[1] for x in fiveletterwords]
l_3 = [x[2] for x in fiveletterwords]
l_4 = [x[3] for x in fiveletterwords]
l_5 = [x[4] for x in fiveletterwords]
all_letters = l_1+l_2+l_3+l_4+l_5

import collections

#count up the number of occurences of each letter in each list
c_1=collections.Counter(l_1)
c_2=collections.Counter(l_2)
c_3=collections.Counter(l_3)
c_4=collections.Counter(l_4)
c_5=collections.Counter(l_5)
alc=collections.Counter(all_letters)

print(sorted(all_letters_count.items(), key=lambda pair: pair[1], reverse=True))

[('a', 4467), ('e', 4255), ('r', 3043), ('o', 2801), ('i', 2581), ('s', 2383), ('t', 2381), ('l', 2368), ('n', 2214), ('u', 1881), ('y', 1605), ('c', 1546), ('d', 1399), ('h', 1323), ('m', 1301), ('p', 1293), ('b', 1162), ('g', 1102), ('k', 882), ('w', 685), ('f', 661), ('v', 466), ('z', 250), ('x', 189), ('j', 163), ('q', 84)]


From this letter count information we can calculate a numerical word score for each 5-letter word in the list.  The word with the highest word score will be our next guess.  One problem that we haven't considered yet is what to do with words which have a repeated letter.  For example the word TENET is composed of letters with a high frequency but only has 3 unique letters and as such doesnt make a good first guess as it gives less overall information than a word like RANCH which has letters with lower frequency counts but no repeated letters.  One way that we can deal with this problem is by multiplying the word score by the number of unique letters in the word for the first couple of rounds of guesses.  A first approach then could be to calculate a word score by adding together the total letter counts of each letter appearing in the word and then multiplying the sum by the number of unique letters in the word.  For our two examples TENET and RANCH this would give:

$$TENET = (2381+4255+2214+4255+2381)*3/5=9291.6 $$
$$RANCH = (3043+4467+2214+1546+1323)*5/5=12593 $$


In [153]:
first_guess_score=[]
for i in fiveletterwords:
    first_guess_score.append((alc[i[0]]+alc[i[1]]+alc[i[2]]+alc[i[3]]+alc[i[4]])*len(set(i))/5)
#sort the wordscores to display the best first guess
firstwordtuple = list(zip(fiveletterwords , first_guess_score ))
sortedwordtuple = sorted(firstwordtuple, key=lambda pair: pair[1], reverse=True)
print(sortedwordtuple[:5])

[('arose', 16949.0), ('orate', 16947.0), ('arise', 16729.0), ('raise', 16729.0), ('serai', 16729.0)]


What about Anagrams however?  Two words which have exactly the same letters but in a different order like SCARE and ACRES.  This word score calculation method gives exactly the same score for two words which are anagrams of one another.  An alternative to the method then would be to use positional letter frequency as that gives us a slightly higher chance of getting green responses.

In [154]:
fgs_anagram=[]
for i in fiveletterwords:
    fgs_anagram.append((c_1[i[0]]+c_2[i[1]]+c_3[i[2]]+c_4[i[3]]+c_5[i[4]])*len(set(i)))
#sort the wordscores to display the best first guess
fwt_anagram = list(zip(fiveletterwords , fgs_anagram ))
swt_anagram = sorted(fwt_anagram, key=lambda pair: pair[1], reverse=True)
print(swt_anagram[:5])

[('sairy', 24930), ('saily', 24925), ('solay', 24500), ('soary', 24455), ('maney', 24260)]


It appears that using the overall letter frequency gives us better initial guesses while using the letter positional frequency may be a better word score calculator for later guesses.  Now that we have a method for ranking our guesses we need imputs to let the algorithm know what word we guessed and what response we got from Wordle for our guess.  To  walk through the algorithm, lets attempt to solve for the Wordle word STEAL, and use as our initial guess the highest score guess word, AROSE.  For this initial guess the first Wordle response would be YXXYY.

In [155]:
#Wordle Guess Input Function
def input_guess():
    while True:
        guess = input("Enter 5-letter word guess: ")
        if len(guess) == 5 and guess.lower() in fiveletterwords:
            break
    return guess.lower()

#Wordle Response Input Function
def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green, Y for Yellow, and X for gray (ex:GGYGX)")
    while True:
        response = input("Color Response from Wordle: ")
        if len(response) == 5 and set(response) <= {"G", "Y", "X"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

guess=input_guess()
responses=input_response()

Enter 5-letter word guess: arose
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Response from Wordle: XGGXX


We can now take this feedback and use it to trim the list of possible solutions to the wordle.  A green response means that we can remove any words that do not have the same letter in the same position, a gray response means that we can remove any words that have that letter, and a yellow response means that we can remove any words that do not have that letter and or do have that letter in that position.  For example in our guess AROSE the A is yellow, therefore we can remove any words that start with A or do not have the letter A in the word.

In [104]:
#Trim solution list using Wordle response information
for i in range(5):
    if responses[i] == 'X':
        fiveletterwords = [x for x in fiveletterwords if guess[i] not in x ]
    elif responses[i] == 'G':
        fiveletterwords = [x for x in fiveletterwords if guess[i] == x[i] ]
    elif responses[i] == 'Y':
        fiveletterwords = [x for x in fiveletterwords if ( guess[i] in x and guess[i] != x[i] )]

#Calculate second guess word scores on the trimmed solution list
second_guess_score=[]
for i in fiveletterwords:
    second_guess_score.append((alc[i[0]]+alc[i[1]]+alc[i[2]]+alc[i[3]]+alc[i[4]])*len(set(i))/5)
#sort the wordscores to display the best first guess
secondwordtuple = list(zip(fiveletterwords , second_guess_score ))
sortedwordtuple = sorted(secondwordtuple, key=lambda pair: pair[1], reverse=True)
print(sortedwordtuple[:5])

[('insea', 15900.0), ('setal', 15854.0), ('steal', 15854.0), ('stela', 15854.0), ('tales', 15854.0)]


After recalculating word scores for the newly trimmed potential solution word list using total letter frequency and penalizing repeated letters again, we have generated a new sorted list of potential guesses.  The guess with the new highest word score which Wordle will accept is SETAL which will give a response to STEAL of: GYYGG.

In [105]:
guess=input_guess()
responses=input_response()

#Trim solution list using Wordle response information
for i in range(5):
    if responses[i] == 'X':
        fiveletterwords = [x for x in fiveletterwords if guess[i] not in x ]
    elif responses[i] == 'G':
        fiveletterwords = [x for x in fiveletterwords if guess[i] == x[i] ]
    elif responses[i] == 'Y':
        fiveletterwords = [x for x in fiveletterwords if ( guess[i] in x and guess[i] != x[i] )]

#Calculate second guess word scores on the trimmed solution list
third_guess_score=[]
for i in fiveletterwords:
    third_guess_score.append((c_1[i[0]]+c_2[i[1]]+c_3[i[2]]+c_4[i[3]]+c_5[i[4]]))
#sort the wordscores to display the best first guess
thirdwordtuple = list(zip(fiveletterwords , third_guess_score ))
sortedwordtuple = sorted(thirdwordtuple, key=lambda pair: pair[1], reverse=True)
print(sortedwordtuple[:5])

Enter 5-letter word guess: Setal
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Response from Wordle: GYYGG
[('steal', 3329)]


Now that we have a basic algorithm that seems to be working there is an additional situation that we haven't yet considered, and that is words with repeated letters.  When a guess has a repeated letter a gray response does not necessarily mean that the repeated letter is not in the solution.  For example consider a guess like TREAT. For the two Ts there are the following potential responses:

Gray + Gray = T is not in the solution word

Gray + Yellow = This is not a viable response, instead Wordle will give Yellow + Gray

Gray + Green = T is the last letter, there is no second T in the solution word

Yellow + Gray = There is exactly 1 T in the solution and it is not the first letter or the last letter

Yellow + Yellow = There are at least 2 T's and they are not the first or the last letters

Yellow + Green = There are at least 2 T's and one of them is the last letter and T is not the first letter

Green + Gray = T is the first letter, there is no second T in the solution word

Green + Yellow = There are at least 2 T's and one of them is the first letter and T is not the last letter

Green + Green = There are at least 2 T's and T is the first and last letter

With our current algorithm a Gray + Green response on a repeated letter T would remove all possible solutions from the solution word list as the first Gray response would remove all words that have the letter T in them and the second Green response would remove all words that do not have the letter T as the last letter which combined remove all words from the list.  In order to avoid this the first thing that we need to do is track when we have a repeated letter guess.  For this example let us use as our guess: TENET with Wordle responses XYYXG.

In [145]:
#Repopulate 5-letter word solution list
fiveletterwords = [x for x in word_list if len(x)==5 and x[0] not in alphabet_list]
#Create Guess with repeated letter
guess="tenet"
responses="XYYXG"
# check for repeated letters
guessletters = [char for char in guess]
repeats = [x for x, count in collections.Counter(guess).items() if count > 1]
repeatsindex=[]
for rl in repeats:
    repeatsindex.append([i for i, j in enumerate(guessletters) if j==rl])
print('repeated letter(s):' , repeats)
print('index of repeated letter(s):' , repeatsindex)
repeatsresponses=[]
for j in repeatsindex:
    repeatsresponses.append([responses[i] for i in j])
print('repeated letter responses:' , repeatsresponses)
repeatresponsedict=dict(zip(repeats , repeatsresponses))
print(repeatresponsedict)
print(repeatresponsedict.get('t'))

repeated letter(s): ['t', 'e']
index of repeated letter(s): [[0, 4], [1, 3]]
repeated letter responses: [['X', 'G'], ['Y', 'X']]
{'t': ['X', 'G'], 'e': ['Y', 'X']}
['X', 'G']


This tells us that the solution word has 1 T and it is the last letter and it has 1 E and it is not the second letter.  Now that we have identified the repeated letters in a guess and the corresponding indexes and responses for these repeated letters we can use this information to correct the response algoritm:

In [149]:
#Trim solution list using Wordle response information corrected with repeated letters:
for i in range(5):
    if guess[i] in repeats:
        #all repeated letters are gray
        if (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==0:
            fiveletterwords = [x for x in fiveletterwords if guess[i] not in x]
        #the current letter is gray and there is exactly 1 non-gray repeated letter
        elif responses[i]=='X' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==1:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i])==1) and x[i] != guess[i]]
        #the current letter is gray and there are exactly 2 non-gray repeated letters
        elif responses[i]=='X' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==2:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] != guess[i]]
        #the current letter is yellow and there are no other repeated letters in the word
        elif responses[i] == 'Y' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 1:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 1) and x[i] != guess[i]]
        #the current letter is yellow and there is exactly 1 other repeated letter in the word
        elif responses[i]=='Y' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==2:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] != guess[i]]
        #the current letter is yellow and there are exactly 2 other repeated letters in the word
        elif responses[i]=='Y' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==3:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 3) and x[i] != guess[i]]
        #the current letter is green and are no other repeated letters in the word
        elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 1:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 1) and x[i] == guess[i]]
        #the current letter is green and there is exactly 1 other repeated letter in the word
        elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 2:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] == guess[i]]
        #the current letter is green and there are exactly 2 other repeated letter in the word
        elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 3:
            fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 3) and x[i] == guess[i]]
    # old rules for non-repeated letter guesses
    elif responses[i] == 'X':
        fiveletterwords = [x for x in fiveletterwords if guess[i] not in x ]
    elif responses[i] == 'G':
        fiveletterwords = [x for x in fiveletterwords if guess[i] == x[i] ]
    elif responses[i] == 'Y':
        fiveletterwords = [x for x in fiveletterwords if ( guess[i] in x and guess[i] != x[i] )]
print(fiveletterwords)        

['agent', 'ament', 'anent', 'arent', 'blent', 'brent', 'enact', 'enapt', 'enhat', 'enrut', 'fient', 'glent', 'inept', 'inert', 'knelt', 'olent', 'scent', 'slent', 'spent', 'urent', 'usent']


For our test case of a guess of TENET with a Wordle Response of XYYXG our algorithm now correctly leaves us with words that end with the letter T, and have exactly one T and one E, and at least 1 N.

# Pulling Everything Together
Now that we have worked out all the wordle response rules we are ready to pull everything together into a single algorithm that we can loop over up to 6 times to play the game.

In [1]:
with open('WordleDict.txt', 'r') as file:
    word_list=[word for line in file for word in line.split()]

import string
alphabet_string = string.ascii_uppercase
alphabet_list = list(alphabet_string)

#Collect all 5 letter non-proper noun words from the dictionary
fiveletterwords = [x for x in word_list if len(x)==5 and x[0] not in alphabet_list]

#collect lists of each letter in each position
l_1 = [x[0] for x in fiveletterwords]
l_2 = [x[1] for x in fiveletterwords]
l_3 = [x[2] for x in fiveletterwords]
l_4 = [x[3] for x in fiveletterwords]
l_5 = [x[4] for x in fiveletterwords]
all_letters = l_1+l_2+l_3+l_4+l_5

import collections

#count up the number of occurences of each letter in each list
c_1=collections.Counter(l_1)
c_2=collections.Counter(l_2)
c_3=collections.Counter(l_3)
c_4=collections.Counter(l_4)
c_5=collections.Counter(l_5)
alc=collections.Counter(all_letters)

#set limit of the number of guesses allowed
maxguessnumber=6

#Define Wordle Guess Input Function
def input_guess():
    while True:
        guess = input("Enter 5-letter word guess: ")
        if len(guess) == 5 and guess.lower() in fiveletterwords:
            break
    return guess.lower()

#Define Wordle Response Input Function
def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green, Y for Yellow, and X for gray (ex:GGYGX)")
    while True:
        response = input("Color Response from Wordle: ")
        if len(response) == 5 and set(response) <= {"G", "Y", "X"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

for k in range(maxguessnumber):
    guess_score=[]
    #for first 2 guesses punish repeated letters and use total letter count
    if k<2:
        for i in fiveletterwords:
            guess_score.append((alc[i[0]]+alc[i[1]]+alc[i[2]]+alc[i[3]]+alc[i[4]])*len(set(i))/5)
    #for remaining guesses dont penalize repeated letters and use positional letter count
    else:
        for i in fiveletterwords:
            guess_score.append((c_1[i[0]]+c_2[i[1]]+c_3[i[2]]+c_4[i[3]]+c_5[i[4]]))
    #sort the wordscores to display the best first guess
    wordtuple = list(zip(fiveletterwords , guess_score ))
    sortedwordtuple = sorted(wordtuple, key=lambda pair: pair[1], reverse=True)
    #Display top 5 guesses
    print(sortedwordtuple[:5])
    #User inputs guess, and Wordle response
    guess=input_guess()
    responses=input_response()
    if responses=='GGGGG':
        print(f'Congrats you solved the Wordle in {k+1} guesses: the Wordle word was {guess}')
        break
    # check for repeated letters
    guessletters = [char for char in guess]
    repeats = [x for x, count in collections.Counter(guess).items() if count > 1]
    repeatsindex=[]
    for rl in repeats:
        repeatsindex.append([i for i, j in enumerate(guessletters) if j==rl])
    repeatsresponses=[]
    for j in repeatsindex:
        repeatsresponses.append([responses[i] for i in j])
    repeatresponsedict=dict(zip(repeats , repeatsresponses))
    #Trim solution list using Wordle response information corrected with repeated letters:
    for i in range(5):
        if guess[i] in repeats:
            #all repeated letters are gray
            if (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==0:
                fiveletterwords = [x for x in fiveletterwords if guess[i] not in x]
            #the current letter is gray and there is exactly 1 non-gray repeated letter
            elif responses[i]=='X' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==1:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i])==1) and x[i] != guess[i]]
            #the current letter is gray and there are exactly 2 non-gray repeated letters
            elif responses[i]=='X' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==2:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] != guess[i]]
            #the current letter is yellow and there are no other repeated letters in the word
            elif responses[i] == 'Y' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 1:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 1) and x[i] != guess[i]]
            #the current letter is yellow and there is exactly 1 other repeated letter in the word
            elif responses[i]=='Y' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==2:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] != guess[i]]
            #the current letter is yellow and there are exactly 2 other repeated letters in the word
            elif responses[i]=='Y' and (repeatresponsedict.get(guess[i]).count('G')+repeatresponsedict.get(guess[i]).count('Y'))==3:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 3) and x[i] != guess[i]]
            #the current letter is green and are no other repeated letters in the word
            elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 1:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 1) and x[i] == guess[i]]
            #the current letter is green and there is exactly 1 other repeated letter in the word
            elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 2:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 2) and x[i] == guess[i]]
            #the current letter is green and there are exactly 2 other repeated letter in the word
            elif responses[i] == 'G' and (repeatresponsedict.get(guess[i]).count('G') + repeatresponsedict.get(guess[i]).count('Y')) == 3:
                fiveletterwords = [x for x in fiveletterwords if (x.count(guess[i]) == 3) and x[i] == guess[i]]
        # old rules for non-repeated letter guesses
        elif responses[i] == 'X':
            fiveletterwords = [x for x in fiveletterwords if guess[i] not in x ]
        elif responses[i] == 'G':
            fiveletterwords = [x for x in fiveletterwords if guess[i] == x[i] ]
        elif responses[i] == 'Y':
            fiveletterwords = [x for x in fiveletterwords if ( guess[i] in x and guess[i] != x[i] )]    

True
[('arose', 16954.0), ('orate', 16952.0), ('arise', 16734.0), ('raise', 16734.0), ('serai', 16734.0)]
Enter 5-letter word guess: orate
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Response from Wordle: YXYXX
[('aliso', 14602.0), ('aloin', 14432.0), ('anoil', 14432.0), ('anoli', 14432.0), ('salon', 14235.0)]
Enter 5-letter word guess: aliso
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Response from Wordle: YXXXY
[('canon', 3669), ('noway', 3615), ('caoba', 3579), ('monad', 3453), ('gonad', 3451)]
Enter 5-letter word guess: canon
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Response from Wordle: GYXYX
[('cooba', 3315), ('couac', 3276), ('couma', 3183), ('cooja', 3179), ('comma', 3025)]
Enter 5-letter word guess: comma
Type the color-coded reply from Wordle:
 G for Green, Y for Yellow, and X for gray (ex:GGYGX)
Color Respo

We now have an algorithm in python to solve for the daily wordle using insights about letter frequency to improve our guessing, in future versions of the algorithm we can apply weight functions to the letter frequencies and iterate over solution lists to allow the algorithm to accurately guess the word in fewer guesses.